In [2]:
stripepath={
        'Zebra': "/nfs/turbo/umms-drjieliu/usr/temp_Fan/Quagga/baselines/StripeCaller/GM12878.bedpe",
        'StripeNN': '/nfs/turbo/umms-drjieliu/usr/temp_Fan/Quagga/baselines/stripenn/GM12878/results_filtered.tsv',
        'Quagga': '/nfs/turbo/umms-drjieliu/proj/Quagga/data/GM12878_bulk/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt'
}

In [53]:
%%bash

awk '{
    if ($4 == 2 || $4 == 6 || $4 == 7) {
        $4 = 1;
        print;
    }
}' /nfs/turbo/umms-drjieliu/proj/ENCODE/ChromHMM/GM12878_GMM_liftover_hg38/E116_15_coreMarks_hg38lift_stateno.bed > ./HMM_anno_peaks/GM12878_HMM_hg38_EPanno_.bed 

In [5]:
def _zebra_read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5=line.split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]))]
                
    return source_dict, hold_dict

In [6]:
def _stripeNN_read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5,l,w,mean,maxpx,pval,stripiness=line.rstrip("\n").split("\t")
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[l,w,mean,maxpx,pval,stripiness]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[l,w,mean,maxpx,pval,stripiness]))]
                
    return source_dict, hold_dict

In [7]:
def _read_stripes(file_name):
    
    hold_dict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}
    source_dict = {5:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]},3:{x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]}}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,_,c4,c5,pval=line.rstrip("\n").split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
            if int(c2)-int(c1)<int(c5)-int(c4):
                source_dict[5][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[float(pval)]))]
            else:
                source_dict[3][chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,c4,c5]]+[float(pval)]))]
                
    return source_dict, hold_dict

In [8]:
def _read_peaks(file_name):
    
    hold_pdict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]+["chrY"]}
    source_pdict = {x:[] for x in [f"chr{n}" for n in range(1,23)] + ["chrX"]+["chrY"]}
    
    check_c = [f"chr{n}" for n in range(1,23)] + ["chrX"] + ["chrY"]
    var5 = []
    var3 = []
    incre=-1
#     with open("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/GM12878.bedpe", "r") as f:
    with open(file_name, "r") as f:
        f.readline()
        for line in f:
            chromo,c1,c2,payload=line.split()
            if chromo != check_c[0]:
                check_c.pop(0)
            incre+=1
#             if int(c2)-int(c1)<int(c5)-int(c4):
            source_pdict[chromo]+=[tuple((incre,chromo,*[int(x) for x in [c1,c2,payload]]))]
                
    return source_pdict, hold_pdict

In [9]:

def IntersectingIntervals(arr1, w_1, arr2, spacer=5000):
    i = j = 0
    n = len(arr1)
    m = len(arr2)
        
    result=[]
    while i < n and j < m:
        while (j < m and arr2[j][3]<arr1[i][w_1]):
            j+=1
            
        if j>=m: #one level up?
            break
        
        if arr1[i][w_1]-spacer<=arr2[j][2]<=arr1[i][w_1+1]+spacer and arr1[i][w_1]-spacer<=arr2[j][3]<=arr1[i][w_1+1]+spacer and arr2[j][-1]!=0:
            result+=[arr1[i]]
            i+=1
        elif arr2[j][2]>arr1[i][w_1+1]+spacer:
            i+=1
        else:
            j+=1
        
    return result

In [10]:
def _load_Intersect(source_dict,source_peaks,hold_dict,spacer=2000):
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        hold_dict[key]=IntersectingIntervals(source_dict[5][key],2,source_peaks[key],spacer=2000)
        hold_dict[key]+=IntersectingIntervals(source_dict[3][key],4,source_peaks[key],spacer=2000)
        
def _write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            payload = value[-1]
            c5=value[-2]
            c4=value[-3]
            write_out+="\t".join([str(y) for y in value[1:4]]+[value[1]]+[str(c4)]+[str(c5)]+[str(payload)])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)

In [11]:
def _z_write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]

            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"
            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)
        
def _snn_write_Intersect(hold_dict, filepath):
    write_out = ""
    for key in [f"chr{n}" for n in range(1,23)] + ["chrX"]:
        for value in hold_dict[key]:
            svalue = [str(x) for x in value]
            write_out+="\t".join([y for y in svalue[1:4]]+[svalue[1]]+[z for z in svalue[4:]])+"\n"            
    with open(filepath, "w") as outfile:
        outfile.write(write_out)

In [12]:
def z_call_CTCF_stripes(stripename, ctcfname, fileoutname, spacer=2000):
    sdict, hdict=_zebra_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
    _z_write_Intersect(hdict, fileoutname)
    
def snn_call_CTCF_stripes(stripename, ctcfname, fileoutname):
    sdict, hdict=_stripeNN_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
    _snn_write_Intersect(hdict, fileoutname)

In [13]:
def call_CTCF_stripes(stripename, ctcfname, fileoutname,spacer=2000):
    sdict, hdict=_read_stripes(stripename)
    source_pdict, hold_pdict = _read_peaks(ctcfname)
    _load_Intersect(sdict, source_pdict,hdict,spacer)
    _write_Intersect(hdict, fileoutname)

In [15]:
!ls /nfs/turbo/umms-drjieliu/usr/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/ctcf_deficient/Othermodels/All_Intersect_EP

Quagga_GM12878_Allstripes_EP.txt


In [13]:
# !mkdir All_Intersect_EP

In [23]:
call_CTCF_stripes("/nfs/turbo/umms-drjieliu/proj/Quagga/data/GM12878_bulk/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt",\
"./HMM_anno_peaks/GM12878_HMM_hg38_EPanno.bed","./All_Intersect_EP/Quagga_GM12878_Allstripes_EP.txt",spacer=5000)

In [ ]:
call_CTCF_stripes("/nfs/turbo/umms-drjieliu/proj/Quagga/data/GM12878_bulk/5000nt_MR10000000_ML50000_MD7000000_MW50000_WS7_SG1RH0.3_NSTR10_ALL_hg38GM12878_completedfiltered.txt",\
"../sorted_GM12878_CTCF.bed","Quagga_GM12878_CTCFonly.txt")

In [21]:
z_call_CTCF_stripes("/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/sorting/sorted_Zebra_GM12878.bedpe",\
                        "../sorted_GM12878_CTCF.bed","Zebra_GM12878_CTCFonly.txt")

In [70]:
z_call_CTCF_stripes("Zebra_GM12878_noCTCF.txt",\
                        "./HMM_anno_peaks/GM12878_HMM_hg38_EPanno.bed","Zebra_GM12878_EP.txt")

In [27]:
snn_call_CTCF_stripes('/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/sorting/sorted_stripeNN_results_filtered.tsv',\
                        "../sorted_GM12878_CTCF.bed","StripeNN_GM12878_CTCFonly.txt")

In [73]:
snn_call_CTCF_stripes("StripeNN_GM12878_noCTCF.txt",\
                        "./HMM_anno_peaks/GM12878_HMM_hg38_EPanno.bed","StripeNN_GM12878_EP.txt")

In [22]:
 z_call_CTCF_stripes("/nfs/turbo/umms-drjieliu/usr/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/AB_zerba_merged_GM12878_3.txt",\
"../sorted_GM12878_CTCF.bed","Zebra_AB_Merged_GM12878_CTCFonly.txt")

In [80]:
snn_call_CTCF_stripes("StripeNN_GM12878_noCTCF_5kspacer.txt",\
                        "./HMM_anno_peaks/GM12878_HMM_hg38_EPanno.bed","StripeNN_GM12878_EP_5kinto2kspacer.txt")

In [81]:
 z_call_CTCF_stripes("Zebra_AB_Merged_GM12878_noCTCF_5kspacer.txt",\
                     "./HMM_anno_peaks/GM12878_HMM_hg38_EPanno.bed","Zebra_AB_Merged_GM12878_EP_5kinto2kspacer.txt")

In [ ]:
z_call_CTCF_stripes('/home/spmoran/temp_smoran/Fan_StripeCaller/2024_SummerPublishPush/zebra_merge/sorting/sorted_stripeNN_results_filtered.tsv',\
                        "../sorted_GM12878_CTCF.bed","StripeNN_GM12878_CTCFonly.txt")

In [86]:
call_EP_stripes("Quagga_GM12878_noCTCF_5kspacer.txt",\
"./HMM_anno_peaks/GM12878_HMM_hg38_EPanno.bed","Quagga_GM12878_EP_5kinto2kspacer.txt")

In [28]:
call_CTCF_stripes("/nfs/turbo/umms-drjieliu/proj/Quagga/data/2024/micro-C/H1_MicroC_2_QuaggaNoIntersectionCTCF.txt",\
"./HMM_anno_peaks/H1_HMM_hg38_EPanno.bed","H1_MicroC_noCTCF_EP.txt",spacer=5000)